In [28]:
%reset -fs
! pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# python 3.12.1

! pip install numpy
! pip install astropy
! pip install astroquery
! pip install matplotlib
! pip install pandas
! pip install scipy
! pip install scikit-learn
! pip install gaiadr3-zeropoint



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Download data from GAIA DR3 database

In [9]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia        

#cluster center
#ra, dec, radius = 290.21375,37.77388888888889,0.38333333333333336 # fov6791_kamann_23
#ra, dec, radius = 295.32166666666666,40.18833333333333,0.38333333333333336 # fov6819_kamann_23
ra = input('Enter RA at cluster center : ')
dec = input('Enter DEC at cluster center : ')
radius = input('Enter Radius in arcmin from the cluster center: ')
 
job = Gaia.launch_job_async("SELECT TOP 10000000 * "
                            "FROM gaiadr3.gaia_source WHERE CONTAINS(POINT('ICRS',gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),"
                            "CIRCLE('ICRS',"+str(ra)+","+str(dec)+","+str(radius)+"))=1",
                            dump_to_file=True, output_format='csv')
r = job.get_results()
print(r)


    solution_id             designation          ... libname_gspphot
------------------- ---------------------------- ... ---------------
1636148068921376768 Gaia DR3 2052799620786165120 ...              --
1636148068921376768 Gaia DR3 2052802090403895552 ...              --
1636148068921376768 Gaia DR3 2052802541363698816 ...              --
1636148068921376768 Gaia DR3 2052802541370131072 ...              --
1636148068921376768 Gaia DR3 2052797353049624064 ...              --
1636148068921376768 Gaia DR3 2052797005151969024 ...              --
1636148068921376768 Gaia DR3 2052797005151969792 ...              --
1636148068921376768 Gaia DR3 2052797005151969920 ...         PHOENIX
1636148068921376768 Gaia DR3 2052797005152486144 ...              --
1636148068921376768 Gaia DR3 2052797005161247360 ...              --
                ...                          ... ...             ...
1636148068921376768 Gaia DR3 2051317062501966080 ...              --
1636148068921376768 Gaia DR3 20513


# Gaussian Mixture Model


Plot the confidence ellipsoids of a mixture of two Gaussians
obtained with Expectation Maximisation (``GaussianMixture`` class) and
Variational Inference (``BayesianGaussianMixture`` class models with
a Dirichlet process prior).

Both models have access to five components with which to fit the data. Note
that the Expectation Maximisation model will necessarily use all five
components while the Variational Inference model will effectively only use as
many as are needed for a good fit. Here we can see that the Expectation
Maximisation model splits some components arbitrarily, because it is trying to
fit too many components, while the Dirichlet Process model adapts it number of
state automatically.

Another advantage of the Dirichlet process model is that it can fit
full covariance matrices effectively even when there are less examples
per cluster than there are dimensions in the data, due to
regularization properties of the inference algorithm.

# Input




In [65]:
# the data downloaded from gaia search module
gaia_data = '1704317857774O-result.csv' # sample data downloaded using the astroquery using above cell.

# could use multiple centroids then we need to find which centroid fits best with the expected results (by looking at the literature values)
# minimum should be 2, one representing cluster and one representing filed star
number_of_centroids = 2 

# The cut of probability that you give to detrmine the centroid parameters from the selected cluster members (0.9 means 90%)
probability_cut_off = 0.9

# if you need include RV filter to probability set to True
rv_parameter = False

# Main module
- Run it after configuring input

In [67]:
%matplotlib
import itertools
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats

from scipy import linalg
from sklearn.cluster import KMeans
from sklearn import mixture

from zero_point import zpt

color_iter = itertools.cycle(['navy', 'c', 'cornflowerblue', 'gold',
                              'darkorange'])


def plot_results(X, Y_, means, covariances, index, title):
    splot = plt.subplot(2, 1, 1 + index)
    for i, (mean, covar, color) in enumerate(zip(
            means, covariances, color_iter)):
        v, w = linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)

    plt.xlim(-9., 5.)
    plt.ylim(-3., 6.)
    plt.xticks(())
    plt.yticks(())
    plt.title(title)

def gauss_mix(X,n,mode,plot=False):
    tic = time.time()
    if mode =='dpgmm':
        # Fit a Dirichlet process Gaussian mixture using five components
        dpgmm = mixture.BayesianGaussianMixture(n_components=n,
                                                covariance_type='full',random_state=1,max_iter=5000).fit(X)
        if plot == True:
            plot_results(X, dpgmm.predict(X), dpgmm.means_, dpgmm.covariances_, 1,
                    'Bayesian Gaussian Mixture with a Dirichlet process prior')

        print('Bayesian Gaussian Mixture with a Dirichlet process prior')
        centers = np.empty(shape=(dpgmm.n_components, X.shape[1]))
        kp = np.zeros(1)
        for i in range(dpgmm.n_components):
            density = scipy.stats.multivariate_normal(cov=dpgmm.covariances_[i], mean=dpgmm.means_[i]).logpdf(X)
            centers[i, :] = X[np.argmax(density)]
            kp = np.vstack((kp,i))
        if plot == True:
            plt.scatter(centers[:, 0], centers[:, 1], s=20,marker='*')
            plt.show()
        print('Full covariance matrices')
        #print(dpgmm.covariances_)
        p=dpgmm.predict_proba(X)
        q=dpgmm.predict(X)
    else:
        # Fit a Gaussian mixture with EM using five components
        gmm = mixture.GaussianMixture(n_components=n, covariance_type='full',random_state=1).fit(X)
        if plot == True:
            plot_results(X, gmm.predict(X), gmm.means_, gmm.covariances_, 0,
                   'Gaussian Mixture')

        print('Gaussian Mixture Results')
        centers = np.empty(shape=(gmm.n_components, X.shape[1]))
        kp = np.zeros(1)
        for i in range(gmm.n_components):
            density = scipy.stats.multivariate_normal(cov=gmm.covariances_[i], mean=gmm.means_[i]).logpdf(X)
            centers[i, :] = X[np.argmax(density)]
            kp = np.vstack((kp,i))
        if plot == True:
            plt.scatter(centers[:, 0], centers[:, 1], s=20,marker='*',color='red')
            plt.show()

        print('Full covariance matrices')
        #print(gmm.covariances_)
        p=gmm.predict_proba(X)
        q=gmm.predict(X)

    if np.shape(X)[1] == 2:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
    elif np.shape(X)[1] == 3:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
    elif np.shape(X)[1] == 4:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
        kp = np.column_stack((kp, centers[:, 3]))
    elif np.shape(X)[1] == 5:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
        kp = np.column_stack((kp, centers[:, 3]))
        kp = np.column_stack((kp, centers[:, 4]))
    elif np.shape(X)[1] == 6:
        kp = np.column_stack((kp[1:],centers[:, 0]))
        kp = np.column_stack((kp, centers[:, 1]))
        kp = np.column_stack((kp, centers[:, 2]))
        kp = np.column_stack((kp, centers[:, 3]))
        kp = np.column_stack((kp, centers[:, 4]))
        kp = np.column_stack((kp, centers[:, 5]))
        
    #print('Centeroids')
    #print(kp)
    toc = time.time()
    print('GMM Run time : ',toc-tic,' seconds')
    return p,q

########### START ####################################################
df1 = pd.read_csv(gaia_data)

df1 = df1[df1.astrometric_params_solved>3] # removing 3 parameter solutions
#zeropoint from lindgren 2020, adsurl = https://ui.adsabs.harvard.edu/abs/2020arXiv201201742L
zpt.load_tables()
df1['zero_point'] = df1.apply(zpt.zpt_wrapper,axis=1)
df1.parallax = df1.parallax - df1.zero_point # zeropoint correction

print('original data: ',len(df1))
df1 = df1[np.isfinite(df1['pmra'])]
df1 = df1[np.isfinite(df1['pmdec'])]
print('after proper-motion nan correction: ',len(df1))
df1 = df1[np.isfinite(df1['parallax'])]
print('after parallax nan correction:', len(df1))
if rv_parameter:
    df1 = df1[np.isfinite(df1['radial_velocity'])]
    print('after radial velocity nan correction:', len(df1))

#uncertainity filter
df1['un_plx'] = abs(df1['parallax_error']/df1['parallax'])*100
df1 = df1[df1.un_plx<=50]
print('after parallax uncertainity filter: ',len(df1))
df1['un_pmra'] = abs(df1['pmra_error']/df1['pmra'])*100
df1 = df1[df1.un_pmra<=50]
print('after pmra uncertainity filter: ',len(df1))
df1['un_pmdec'] = abs(df1['pmdec_error']/df1['pmdec'])*100
df1 = df1[df1.un_pmdec<=50]
print('after pmdec uncertainity filter: ',len(df1))

#filtering forground and background
df1 = df1[(df1.parallax>0)&(df1.parallax<1)]
print('after parallax field filter: ',len(df1))

print('RUNNING GMM MODELS ON THE PREPROCESSED DATA...')

p1 = df1.pmra
p2 = df1.pmdec
p3 = df1.parallax
if rv_parameter:
    p4 = df1.radial_velocity
p5 = df1.ra
p6 = df1.dec

X = np.column_stack([p1,p2])
X = np.column_stack([X,p3])
if rv_parameter:
    X = np.column_stack([X,p4])
X = np.column_stack([X,p5])
X = np.column_stack([X,p6])

XX = np.column_stack([df1.designation,X])
print('Final data length before gmm : ', len(df1))
#GMM
p,q = gauss_mix(X,int(number_of_centroids),'dpgmm')

#choose the right centroid from priors
for i in range(number_of_centroids):
    c = i        
    #Stacking Data
    gdata = np.column_stack([XX,q])
    pb = np.zeros(len(p))
    avgp1 = np.zeros(len(p))
    avgp2 = np.zeros(len(p))
    A = 'ID,GM,P,gmm_prob'
    gmm_prob = np.zeros(1)
    for j in range(len(p)):
        pb[j] = p[j][q[j]]
        gmm_prob = np.vstack((gmm_prob,p[j][int(c)]))
    gdata = np.column_stack([gdata,pb])
    df1[rf'gmm_prob_centroid_{i}'] = gmm_prob[1:]


df1.to_csv(rf'GMM_PROBABILITY.csv')

print('displaying the paramters and plots for each centroids...')

for i in range(number_of_centroids):
    pt = probability_cut_off
    print('Almost certain members : ', len(df1[df1[rf'gmm_prob_centroid_{i}']>pt]))
    dfc = df1[df1[rf'gmm_prob_centroid_{i}']>pt]
    ldata = len(dfc)
    p1 = dfc.pmra
    p2 = dfc.pmdec
    p3 = dfc.parallax
    p4 = dfc.ra
    p5 = dfc.dec
    if rv_parameter:
        p6 = dfc.radial_velocity
    
    plt.figure(figsize=(10,6))
    plt.rc('xtick', labelsize=12)
    plt.rc('ytick', labelsize=12)
    plt.plot(dfc.bp_rp,dfc.phot_g_mean_mag,'.',ms=2,color='grey',label=r'cluster members')
    plt.gca().invert_yaxis()
    plt.legend(loc=2,frameon=False)
    plt.ylabel(r'G',size = 12)
    
    if rv_parameter:
        titl = rf'''Centroid {i}, ra: {np.mean(p5)} +/- {np.std(p5)/ldata}, dec: {np.mean(p4)} +/- {np.std(p4)/ldata}, 
            pmra: {np.mean(p1)} +/- {np.std(p1)/ldata}, pmdec: {np.mean(p2)} +/- {np.std(p2)/ldata},
            parallax: {np.mean(p3)} +/- {np.std(p3)/ldata}, radial_velocity: {np.mean(p6)} +/- {np.std(p6)/ldata}'''
    else:
        titl = rf'''Centroid {i}, ra: {np.mean(p5)} +/- {np.std(p5)/ldata}, dec: {np.mean(p4)} +/- {np.std(p4)/ldata}, 
            pmra: {np.mean(p1)} +/- {np.std(p1)/ldata}, pmdec: {np.mean(p2)} +/- {np.std(p2)/ldata},
            parallax: {np.mean(p3)} +/- {np.std(p3)/ldata}'''
    
    plt.title(titl)
    plt.tight_layout()
    plt.savefig(rf'centroid_{i}.png')
    plt.show()
    print(titl)
    

Using matplotlib backend: TkAgg
original data:  33217
after proper-motion nan correction:  33217
after parallax nan correction: 29920
after parallax uncertainity filter:  14357
after pmra uncertainity filter:  13475
after pmdec uncertainity filter:  13228
after parallax field filter:  11306
RUNNING GMM MODELS ON THE PREPROCESSED DATA...
Final data length before gmm :  11306
Bayesian Gaussian Mixture with a Dirichlet process prior
Full covariance matrices
GMM Run time :  0.262270450592041  seconds
displaying the paramters and plots for each centroids...
Almost certain members :  2177
Centroid 0, ra: 37.77640230153851 +/- 2.9743664460028377e-05, dec: 290.22124364664603 +/- 3.657001616541952e-05, 
            pmra: -0.4320789509764506 +/- 4.818479559396929e-05, pmdec: -2.2813222630219423 +/- 5.591265328930792e-05,
            parallax: 0.27565330447926045 +/- 3.547134166988586e-05
Almost certain members :  8979
Centroid 1, ra: 37.7659574834137 +/- 2.0980710404155247e-05, dec: 290.21921503

# NOTES
- The output is saved as 'GMM_PROBABILITY.csv'
- It will also plot the results from differnt centroids in CMD.
- The probability parameters obtained from different centroids are given as outputs.
- If you choose one centroid looking at the CMD and the parameter values, you can find the probability column in the csv file with name 'gmm_prob_centroid_x' 